In [1]:
!nvidia-smi
#https://stackoverflow.com/questions/49311195/how-to-install-coco-pythonapi-in-python3

Mon May  9 12:34:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1B.0 Off |                    0 |
| N/A   22C    P8     8W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            On   | 00000000:00:1C.0 Off |                    0 |
| N/A   

In [1]:
%%capture
!pip install kaggle
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


In [2]:
! kaggle datasets download -d benceveges/yolo-flir

100%|██████████████████████████████████████▉| 6.58G/6.58G [01:06<00:00, 176MB/s]
100%|███████████████████████████████████████| 6.58G/6.58G [01:06<00:00, 107MB/s]


In [3]:
%%capture
!unzip yolo-flir.zip

In [4]:
!git clone https://github.com/SudharsanamShyamsundar/FLIR_YOLOR.git

Cloning into 'FLIR_YOLOR'...
remote: Enumerating objects: 2432, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 2432 (delta 1), reused 8 (delta 1), pack-reused 2424
Receiving objects: 100% (2432/2432), 444.35 MiB | 70.26 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [5]:
import os
import shutil
#Thermal Train images
for file_name in os.listdir('/dli/task/images_thermal_train/images_thermal_train/'):
    # construct full file path
    source = '/dli/task/images_thermal_train/images_thermal_train/' + file_name
    destination = '/dli/task/FLIR_YOLOR/flir_f1/images/train/' + file_name
    # move only files
    if os.path.isfile(source):
        shutil.move(source, destination)

# thermal train label
for file_name in os.listdir('/dli/task/labels/coco_thermal_train/'):
    # construct full file path
    source = '/dli/task/labels/coco_thermal_train/' + file_name
    destination = '/dli/task/FLIR_YOLOR/flir_f1/labels/train/' + file_name
    # move only files
    if os.path.isfile(source):
        shutil.move(source, destination)

# Thermal val images
for file_name in os.listdir('/dli/task/images_thermal_val/images_thermal_val/'):
    # construct full file path
    source = '/dli/task/images_thermal_val/images_thermal_val/' + file_name
    destination = '/dli/task/FLIR_YOLOR/flir_f1/images/val/' + file_name
    # move only files
    if os.path.isfile(source):
        shutil.move(source, destination)

# thermal validation label set
for file_name in os.listdir('/dli/task/labels/coco_thermal_val/'):
    # construct full file path
    source = '/dli/task/labels/coco_thermal_val/' + file_name
    destination = '/dli/task/FLIR_YOLOR/flir_f1/labels/val/' + file_name
    # move only files
    if os.path.isfile(source):
        shutil.move(source, destination)
        
# Thermal test images
for file_name in os.listdir('/dli/task/video_thermal_test/video_thermal_test/'):
    # construct full file path
    source = '/dli/task/video_thermal_test/video_thermal_test/' + file_name
    destination = '/dli/task/FLIR_YOLOR/flir_f1/images/test/' + file_name
    # move only files
    if os.path.isfile(source):
        shutil.move(source, destination)

# thermal test label set
for file_name in os.listdir('/dli/task/labels/coco_thermal_test/'):
    # construct full file path
    source = '/dli/task/labels/coco_thermal_test/' + file_name
    destination = '/dli/task/FLIR_YOLOR/flir_f1/labels/test/' + file_name
    # move only files
    if os.path.isfile(source):
        shutil.move(source, destination)

In [6]:
%cd /dli/task/FLIR_YOLOR

/dli/task/FLIR_YOLOR


In [7]:
!pip install -qr requirements.txt

In [8]:
!git clone https://github.com/JunnYu/mish-cuda
%cd mish-cuda

Cloning into 'mish-cuda'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 195 (delta 7), reused 79 (delta 3), pack-reused 107
Receiving objects: 100% (195/195), 208.77 KiB | 23.20 MiB/s, done.
Resolving deltas: 100% (56/56), done.
/dli/task/FLIR_YOLOR/mish-cuda


In [9]:
%%capture
!python setup.py build install
%cd ..

In [10]:
%%capture
!git clone https://github.com/fbcotter/pytorch_wavelets
%cd pytorch_wavelets
!pip install .
%cd ..

In [ ]:
#Open Cv libraries are not properly installed and hence we are using this below command to force install libaries
!apt-get install ffmpeg libsm6 libxext6  -y


In [ ]:
#We dont need to run this , as we have more cores in hand 
#!python train.py --batch-size 4 --img 1280 1280 --data flir.yaml --cfg cfg/yolor_p6.cfg --weights '' --device 0 --name yolor_p6 --hyp '/dli/task/FLIR_YOLOR/data/hyp.finetune.1280.yaml' --epochs 1

In [13]:
!python -m torch.distributed.launch --nproc_per_node 4 --master_port 9527 train.py --batch-size 24 --img 1280 1280 --data flir.yaml --cfg cfg/yolor_p6.cfg --weights '' --device 0,1,2,3 --sync-bn --name yolor_p6 --hyp '/dli/task/FLIR_YOLOR/data/hyp.scratch.1280.yaml' --epochs 10


*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
Using torch 1.7.0 CUDA:0 (Tesla T4, 15109MB)
                  CUDA:1 (Tesla T4, 15109MB)
                  CUDA:2 (Tesla T4, 15109MB)
                  CUDA:3 (Tesla T4, 15109MB)

Namespace(adam=False, batch_size=6, bucket='', cache_images=False, cfg='cfg/yolor_p6.cfg', data='flir.yaml', device='0,1,2,3', epochs=10, evolve=False, exist_ok=False, global_rank=0, hyp='/dli/task/FLIR_YOLOR/data/hyp.scratch.1280.yaml', image_weights=False, img_size=[1280, 1280], local_rank=0, log_imgs=16, multi_scale=False, name='yolor_p6', noautoanchor=False, nosave=False, notest=False, project='runs/train', rect=False, resume=False, save_dir='runs/train/yolor_p62', single_cls=False, sync_bn=True, total_batch_size=24

In [14]:
!python detect.py --weights /dli/task/FLIR_YOLOR/runs/train/yolor_p62/weights/last.pt --conf 0.25 --source /dli/task/FLIR_YOLOR/flir_f1/images/test/video-4FRnNpmSmwktFJKjg-frame-000745-L6K5SC6fYjHNC8uff.jpg --names /dli/task/FLIR_YOLOR/data/coco.names --cfg cfg/yolor_p6.cfg

Namespace(agnostic_nms=False, augment=False, cfg='cfg/yolor_p6.cfg', classes=None, conf_thres=0.25, device='', img_size=1280, iou_thres=0.5, names='/dli/task/FLIR_YOLOR/data/coco.names', output='inference/output', save_txt=False, source='/dli/task/FLIR_YOLOR/flir_f1/images/test/video-4FRnNpmSmwktFJKjg-frame-000745-L6K5SC6fYjHNC8uff.jpg', update=False, view_img=False, weights=['/dli/task/FLIR_YOLOR/runs/train/yolor_p62/weights/last.pt'])
image 1/1 /dli/task/FLIR_YOLOR/flir_f1/images/test/video-4FRnNpmSmwktFJKjg-frame-000745-L6K5SC6fYjHNC8uff.jpg: 1024x1280 1 persons, 1 cars, Done. (0.060s)
Results saved to inference/output
Done. (0.154s)


In [15]:
!python test.py --data flir.yaml --img 1280 --batch 32 --conf 0.001 --iou 0.65 --device 0 --cfg cfg/yolor_p6.cfg --weights /dli/task/FLIR_YOLOR/runs/train/yolor_p62/weights/last.pt --name yolor_p6_val

Namespace(augment=False, batch_size=32, cfg='cfg/yolor_p6.cfg', conf_thres=0.001, data='flir.yaml', device='0', exist_ok=False, img_size=1280, iou_thres=0.65, name='yolor_p6_val', names='data/coco.names', project='runs/test', save_conf=False, save_json=False, save_txt=False, single_cls=False, task='val', verbose=False, weights=['/dli/task/FLIR_YOLOR/runs/train/yolor_p62/weights/last.pt'])
Using torch 1.7.0 CUDA:0 (Tesla T4, 15109MB)

Model Summary: 665 layers, 37265016 parameters, 37265016 gradients, 81.564040600 GFLOPS
Scanning labels /dli/task/FLIR_YOLOR/flir_f1/labels/val.cache3 (1128 found, 0 missing, 0 empty, 2 duplicate, for 1128 images): 1128it [00:00, 17278.16it/s]
               Class      Images     Targets           P           R      mAP@.5
                 all    1.13e+03    1.67e+04       0.301       0.391       0.328       0.176
Speed: 41.4/2.6/44.0 ms inference/NMS/total per 1280x1280 image at batch-size 32
Results saved to runs/test/yolor_p6_val


In [ ]:
#Open Cv libraries are not properly installed and hence we are using this below command to force install libaries
%%capture
!apt-get install ffmpeg libsm6 libxext6  -y
